In [ ]:
import torch
from torch.utils.data import Dataset
import os
import pickle
from collections import defaultdict
import pandas as pd
import numpy as np
import gzip
import shutil
import spacy 
import pandas as pd 
from tqdm import tqdm

In [ ]:
!pip install kora -q

     |████████████████████████████████| 57 kB 3.8 MB/s 
     |████████████████████████████████| 60 kB 7.2 MB/s 


In [ ]:
import kora.install.rdkit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw, MolFromSmiles, MolToSmiles

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
!cp -r "/content/drive/MyDrive/11785FinalProject" /content/

%cd /content/11785FinalProject
!pwd

/content/drive/.shortcut-targets-by-id/1dBDfrLzsluViEo50WrncEp56WUihj3GB/11785FinalProject
/content/drive/.shortcut-targets-by-id/1dBDfrLzsluViEo50WrncEp56WUihj3GB/11785FinalProject


In [ ]:
# !unzip -q /content/drive/MyDrive/Colab Notebooks/11785FinalProject/transformer_xl.zip

In [ ]:
!ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
 0411smilesmodel_epoch0.pth
 bace.csv
 BBBP.csv
 chembl_24_chemreps.txt.gz
 chembl_24_dropped.csv
 chembl_30_chemreps.txt.gz
 ChemBL_Explore.ipynb
 dataset
'Deep Learning Project Proposal.docx'
'de novo Molecular Generation using SMILES Transformer.gdoc'
'de novo Molecular Generation using SMILES Transformer.pdf'
 file.txt
 HIV.csv
 Load_baseline_model.ipynb
 __MACOSX
 Manuscript
 MITCourse.pdf
 notebooks
 PlotComparison.ipynb
 smilesmodel_epoch0.pth
 smilesmodel_epoch1.pth
 smilesmodel_epoch2.pth
 smilesmodel_epoch3.pth
 smilesmodel_epoch4.pth
 smilesmodel_epoch5.pth
 smilesmodel_epoch6.pth
 smilesmodel_epoch7.pth
 smilesmodel_epoch8.pth
 smilesmodel_epoch9.pth
 transformer_xl
 transformer_xl.zip
 trfm_12_23000.pkl
 vocab24.pkl
 vocab_chembl24.pkl
 vocab.pkl


In [ ]:
# reproducibility
torch.manual_seed(314)
np.random.seed(314)

# Smile Enumerator

In [ ]:
class Iterator(object):
    """Abstract base class for data iterators.
    # Arguments
        n: Integer, total number of samples in the dataset to loop over.
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seeding for data shuffling.
    """

    def __init__(self, n, batch_size, shuffle, seed):
        self.n = n
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.batch_index = 0
        self.total_batches_seen = 0
        self.index_generator = self._flow_index(n, batch_size, shuffle, seed)
        if n < batch_size:
            raise ValueError('Input data length is shorter than batch_size\nAdjust batch_size')

    def reset(self):
        self.batch_index = 0

    def _flow_index(self, n, batch_size=32, shuffle=False, seed=None):
        # Ensure self.batch_index is 0.
        self.reset()
        while 1:
            if seed is not None:
                np.random.seed(seed + self.total_batches_seen)
            if self.batch_index == 0:
                index_array = np.arange(n)
                if shuffle:
                    index_array = np.random.permutation(n)

            current_index = (self.batch_index * batch_size) % n
            if n > current_index + batch_size:
                current_batch_size = batch_size
                self.batch_index += 1
            else:
                current_batch_size = n - current_index
                self.batch_index = 0
            self.total_batches_seen += 1
            yield (index_array[current_index: current_index + current_batch_size],
                   current_index, current_batch_size)

    def __iter__(self):
        # Needed if we want to do something like:
        # for x, y in data_gen.flow(...):
        return self

    def __next__(self, *args, **kwargs):
        return self.next(*args, **kwargs)




class SmilesIterator(Iterator):
    """Iterator yielding data from a SMILES array.
    # Arguments
        x: Numpy array of SMILES input data.
        y: Numpy array of targets data.
        smiles_data_generator: Instance of `SmilesEnumerator`
            to use for random SMILES generation.
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seed for data shuffling.
        dtype: dtype to use for returned batch. Set to keras.backend.floatx if using Keras
    """

    def __init__(self, x, y, smiles_data_generator,
                 batch_size=32, shuffle=False, seed=None,
                 dtype=np.float32
                 ):
        if y is not None and len(x) != len(y):
            raise ValueError('X (images tensor) and y (labels) '
                             'should have the same length. '
                             'Found: X.shape = %s, y.shape = %s' %
                             (np.asarray(x).shape, np.asarray(y).shape))

        self.x = np.asarray(x)

        if y is not None:
            self.y = np.asarray(y)
        else:
            self.y = None
        self.smiles_data_generator = smiles_data_generator
        self.dtype = dtype
        super(SmilesIterator, self).__init__(x.shape[0], batch_size, shuffle, seed)

    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x = np.zeros(tuple([current_batch_size] + [ self.smiles_data_generator.pad, self.smiles_data_generator._charlen]), dtype=self.dtype)
        for i, j in enumerate(index_array):
            smiles = self.x[j:j+1]
            x = self.smiles_data_generator.transform(smiles)
            batch_x[i] = x

        if self.y is None:
            return batch_x
        batch_y = self.y[index_array]
        return batch_x, batch_y


class SmilesEnumerator(object):
    """SMILES Enumerator, vectorizer and devectorizer
    
    #Arguments
        charset: string containing the characters for the vectorization
          can also be generated via the .fit() method
        pad: Length of the vectorization
        leftpad: Add spaces to the left of the SMILES
        isomericSmiles: Generate SMILES containing information about stereogenic centers
        enum: Enumerate the SMILES during transform
        canonical: use canonical SMILES during transform (overrides enum)
    """
    def __init__(self, charset = '@C)(=cOn1S2/H[N]\\', pad=120, leftpad=True, isomericSmiles=True, enum=True, canonical=False):
        self._charset = None
        self.charset = charset
        self.pad = pad
        self.leftpad = leftpad
        self.isomericSmiles = isomericSmiles
        self.enumerate = enum
        self.canonical = canonical

    @property
    def charset(self):
        return self._charset
        
    @charset.setter
    def charset(self, charset):
        self._charset = charset
        self._charlen = len(charset)
        self._char_to_int = dict((c,i) for i,c in enumerate(charset))
        self._int_to_char = dict((i,c) for i,c in enumerate(charset))
        
    def fit(self, smiles, extra_chars=[], extra_pad = 5):
        """Performs extraction of the charset and length of a SMILES datasets and sets self.pad and self.charset
        
        #Arguments
            smiles: Numpy array or Pandas series containing smiles as strings
            extra_chars: List of extra chars to add to the charset (e.g. "\\\\" when "/" is present)
            extra_pad: Extra padding to add before or after the SMILES vectorization
        """
        charset = set("".join(list(smiles)))
        self.charset = "".join(charset.union(set(extra_chars)))
        self.pad = max([len(smile) for smile in smiles]) + extra_pad
        
    def randomize_smiles(self, smiles):
        """Perform a randomization of a SMILES string
        must be RDKit sanitizable"""
        m = Chem.MolFromSmiles(smiles)
        if m is None:
          return None
        ans = list(range(m.GetNumAtoms()))
        np.random.shuffle(ans)
        nm = Chem.RenumberAtoms(m,ans)
        return Chem.MolToSmiles(nm, canonical=self.canonical, isomericSmiles=self.isomericSmiles)

    def transform(self, smiles):
        """Perform an enumeration (randomization) and vectorization of a Numpy array of smiles strings
        #Arguments
            smiles: Numpy array or Pandas series containing smiles as strings
        """
        one_hot =  np.zeros((smiles.shape[0], self.pad, self._charlen),dtype=np.int8)
        
        if self.leftpad:
            for i,ss in enumerate(smiles):
                if self.enumerate: ss = self.randomize_smiles(ss)
                l = len(ss)
                diff = self.pad - l
                for j,c in enumerate(ss):
                    one_hot[i,j+diff,self._char_to_int[c]] = 1
            return one_hot
        else:
            for i,ss in enumerate(smiles):
                if self.enumerate: ss = self.randomize_smiles(ss)
                for j,c in enumerate(ss):
                    one_hot[i,j,self._char_to_int[c]] = 1
            return one_hot

      
    def reverse_transform(self, vect):
        """ Performs a conversion of a vectorized SMILES to a smiles strings
        charset must be the same as used for vectorization.
        #Arguments
            vect: Numpy array of vectorized SMILES.
        """       
        smiles = []
        for v in vect:
            #mask v 
            v=v[v.sum(axis=1)==1]
            #Find one hot encoded index with argmax, translate to char and join to string
            smile = "".join(self._int_to_char[i] for i in v.argmax(axis=1))
            smiles.append(smile)
        return np.array(smiles)
     
# if __name__ == "__main__":
#     smiles = np.array([ "CCC(=O)O[C@@]1(CC[NH+](C[C@H]1CC=C)C)c2ccccc2",
#                         "CCC[S@@](=O)c1ccc2c(c1)[nH]/c(=N/C(=O)OC)/[nH]2"]*10
#                         )
#     #Test canonical SMILES vectorization
#     sm_en = SmilesEnumerator(canonical=True, enum=False)
#     sm_en.fit(smiles, extra_chars=["\\"])
#     v = sm_en.transform(smiles)
#     transformed = sm_en.reverse_transform(v)
#     if len(set(transformed)) > 2: print ("Too many different canonical SMILES generated")
    
#     #Test enumeration 
#     sm_en.canonical = False
#     sm_en.enumerate = True
#     v2 = sm_en.transform(smiles)
#     transformed = sm_en.reverse_transform(v2)
#     if len(set(transformed)) < 3: print ("Too few enumerated SMILES generated")

#     #Reconstruction
#     reconstructed = sm_en.reverse_transform(v[0:5])
#     for i, smile in enumerate(reconstructed):
#         if smile != smiles[i]:
#             print ("Error in reconstruction %s %s".format(smile, smiles[i]))
#             break
    
#     #test Pandas
#     import pandas as pd
#     df = pd.DataFrame(smiles)
#     v = sm_en.transform(df[0])
#     if v.shape != (20, 52, 18): print ("Possible error in pandas use")
    
#     #BUG, when batchsize > x.shape[0], then it only returns x.shape[0]!
#     #Test batch generation
#     sm_it = SmilesIterator(smiles, np.array([1,2]*10), sm_en, batch_size=10, shuffle=True)
#     X, y = sm_it.next()
#     if sum(y==1) - sum(y==2) > 1:
#         print ("Unbalanced generation of batches")
#     if len(X) != 10: print ("Error in batchsize generation" )

# Data Loader

In [ ]:
class Randomizer(object):
    def __init__(self):
        self.enum = SmilesEnumerator()
    
    def __call__(self, smile):
        random_smile = self.enum.randomize_smiles(smile)
        if random_smile is None:
            sm_spaced = split(smile)
        else:
            sm_spaced = split(random_smile)
        sm_split = sm_spaced.split()

        if len(sm_split) <= 218:
            return sm_split
        else:
            return split(sm).split()

    def random_transform(self, sm):
        return self.enum.randomize_smiles(sm)

# Copied Code
class Seq2seqDataset(Dataset):

    def __init__(self, smiles, vocab, seq_len=220, transform=Randomizer()):
        self.smiles = smiles
        self.vocab = vocab
        self.seq_len = seq_len
        self.transform = transform

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, item):
        sm = self.smiles[item]
        if sm is None:
          return None
        sm = self.transform(sm)
        content = [self.vocab.stoi.get(token, self.vocab.unk_index) for token in sm]
        X = [self.vocab.sos_index] + content + [self.vocab.eos_index]
        padding = [self.vocab.pad_index]*(self.seq_len - len(X))
        X.extend(padding)
        return torch.tensor(X)

Utils Library

In [ ]:
import torch
import math
import torch.nn as nn
from rdkit import Chem
from rdkit import rdBase
rdBase.DisableLog('rdApp.*')

def split(sm):
    sm_couples = []
    i = 0
    while i < len(sm):
        if i == len(sm) - 1:
            sm_couples.append(sm[i])
            i += 1
        elif sm[i] == '%':
            sm_couples.append(sm[i:i+3])
            i += 3
        elif sm[i:i+2] in ['Cl', 'Ca', 'Cu', 'Br', 'Be', 'Ba', 'Bi', 'Si', 'Se', 'Sr', 'Na', 'Ni', 'Rb', 'Ra', 'Xe', 'Li', 'Al', ' As', 'Ag', 'Au',\
                            'Mg', 'Mn', 'Te', 'Zn', 'si', 'se', 'te', 'He', 'Kr', 'Fe', '+2', '+3', '+4', '-2', '-3', '-4']:
            sm_couples.append(sm[i:i+2])
            i += 2
        else:
            sm_couples.append(sm[i])
            i += 1
    return ' '.join(sm_couples)
# 活性化関数
# class GELU(nn.Module):
#     def forward(self, x):
#         return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

# 位置情報を考慮したFFN
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.GELU()

    def forward(self, x):
        return self.w_2(self.dropout(self.activation(self.w_1(x))))

Build Vocab

In [ ]:
import argparse
import pickle
from collections import Counter


class TorchVocab(object):
    def __init__(self, counter, max_size=None, min_freq=1, specials=['<pad>', '<oov>'],
                 vectors=None, unk_init=None, vectors_cache=None):
        self.freqs = counter
        counter = counter.copy()
        min_freq = max(min_freq, 1)

        self.itos = list(specials)
        # special tokensの出現頻度はvocabulary作成の際にカウントされない
        for tok in specials:
            del counter[tok]

        max_size = None if max_size is None else max_size + len(self.itos)

        # まず頻度でソートし、次に文字順で並び替える
        words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
        words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)
        
        # 出現頻度がmin_freq未満のものはvocabに加えない
        for word, freq in words_and_frequencies:
            if freq < min_freq or len(self.itos) == max_size:
                break
            self.itos.append(word)

        # dictのk,vをいれかえてstoiを作成する
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}

        self.vectors = None
        if vectors is not None:
            self.load_vectors(vectors, unk_init=unk_init, cache=vectors_cache)
        else:
            assert unk_init is None and vectors_cache is None

    def __eq__(self, other):
        if self.freqs != other.freqs:
            return False
        if self.stoi != other.stoi:
            return False
        if self.itos != other.itos:
            return False
        if self.vectors != other.vectors:
            return False
        return True

    def __len__(self):
        return len(self.itos)

    def vocab_rerank(self):
        self.stoi = {word: i for i, word in enumerate(self.itos)}

    def extend(self, v, sort=False):
        words = sorted(v.itos) if sort else v.itos
        for w in words:
            if w not in self.stoi:
                self.itos.append(w)
                self.stoi[w] = len(self.itos) - 1


class Vocab(TorchVocab):
    def __init__(self, counter, max_size=None, min_freq=1):
        self.pad_index = 0
        self.unk_index = 1
        self.eos_index = 2
        self.sos_index = 3
        self.mask_index = 4
        super().__init__(counter, specials=["<pad>", "<unk>", "<eos>", "<sos>", "<mask>"], max_size=max_size, min_freq=min_freq)

    # override用
    def to_seq(self, sentece, seq_len, with_eos=False, with_sos=False) -> list:
        pass

    # override用
    def from_seq(self, seq, join=False, with_pad=False):
        pass

    @staticmethod
    def load_vocab(vocab_path: str) -> 'Vocab':
        with open(vocab_path, "rb") as f:
            return pickle.load(f)

    def save_vocab(self, vocab_path):
        with open(vocab_path, "wb") as f:
            pickle.dump(self, f)


# テキストファイルからvocabを作成する
class WordVocab(Vocab):
    def __init__(self, texts, max_size=None, min_freq=1):
        print("Building Vocab")
        counter = Counter()
        for line in texts:
            if isinstance(line, list):
                words = line
            else:
                words = line.replace("\n", "").replace("\t", "").split()

            for word in words:
                counter[word] += 1
        super().__init__(counter, max_size=max_size, min_freq=min_freq)

    def to_seq(self, sentence, seq_len=None, with_eos=False, with_sos=False, with_len=False):
        if isinstance(sentence, str):
            sentence = sentence.split()

        seq = [self.stoi.get(word, self.unk_index) for word in sentence]

        if with_eos:
            seq += [self.eos_index]  # this would be index 1
        if with_sos:
            seq = [self.sos_index] + seq

        origin_seq_len = len(seq)

        if seq_len is None:
            pass
        elif len(seq) <= seq_len:
            seq += [self.pad_index for _ in range(seq_len - len(seq))]
        else:
            seq = seq[:seq_len]

        return (seq, origin_seq_len) if with_len else seq

    def from_seq(self, seq, join=False, with_pad=False):
        words = [self.itos[idx]
                 if idx < len(self.itos)
                 else "<%d>" % idx
                 for idx in seq
                 if not with_pad or idx != self.pad_index]

        return " ".join(words) if join else words

    @staticmethod
    def load_vocab(vocab_path: str) -> 'WordVocab':
        with open(vocab_path, "rb") as f:
            return pickle.load(f)


# def main():
#     parser = argparse.ArgumentParser(description='Build a vocabulary pickle')
#     parser.add_argument('--corpus_path', '-c', type=str, default='data/chembl24_corpus.txt', help='path to th ecorpus')
#     parser.add_argument('--out_path', '-o', type=str, default='data/vocab.pkl', help='output file')
#     parser.add_argument('--min_freq', '-m', type=int, default=500, help='minimum frequency for vocabulary')
#     parser.add_argument('--vocab_size', '-v', type=int, default=None, help='max vocabulary size')
#     parser.add_argument('--encoding', '-e', type=str, default='utf-8', help='encoding of corpus')
#     args = parser.parse_args()

#     with open(args.corpus_path, "r", encoding=args.encoding) as f:
#         vocab = WordVocab(f, max_size=args.vocab_size, min_freq=args.min_freq)

#     print("VOCAB SIZE:", len(vocab))
#     vocab.save_vocab(args.out_path)

#if __name__=='__main__':
#    main()
     


In [ ]:
vocab = WordVocab.load_vocab('/content/drive/MyDrive/data/vocab24.pkl')
print(len(vocab))
dir(vocab)

45


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'eos_index',
 'extend',
 'freqs',
 'from_seq',
 'itos',
 'load_vocab',
 'mask_index',
 'pad_index',
 'save_vocab',
 'sos_index',
 'stoi',
 'to_seq',
 'unk_index',
 'vectors',
 'vocab_rerank']

In [ ]:
import copy
from typing import Optional, Any, Union, Callable

import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F

class SandwichTransformer(nn.Module):

    def __init__(self, d_model = 512, nhead: int = 8, num_encoder_layers: int = 6,
                 num_decoder_layers: int = 6, dim_feedforward: int = 2048, dropout: float = 0.1,
                 sandwich_k = 2, sandwich_encoder = False, sandwich_decoder = False,
                 activation: Union[str, Callable[[Tensor], Tensor]] = F.relu,
                 custom_encoder: Optional[Any] = None, custom_decoder: Optional[Any] = None,
                 layer_norm_eps: float = 1e-5, batch_first: bool = False, norm_first: bool = False,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(SandwichTransformer, self).__init__()

        encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout,
                                                activation, layer_norm_eps, batch_first, norm_first,
                                                **factory_kwargs)
        encoder_norm = nn.LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm, sandwich_k if sandwich_encoder else 0)

        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout,
                                                activation, layer_norm_eps, batch_first, norm_first,
                                                **factory_kwargs)
        decoder_norm = nn.LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.decoder = TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm, sandwich_k if sandwich_decoder else 0)

        self._reset_parameters()

        self.d_model = d_model
        self.nhead = nhead

        self.batch_first = batch_first
        # self.sandwich_k = sandwich_k
        # self.sandwich_encoder = sandwich_encoder
        # self.sandwich_decoder = sandwich_decoder

    def forward(self, src: Tensor, tgt: Tensor, src_mask: Optional[Tensor] = None, tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None, memory_key_padding_mask: Optional[Tensor] = None) -> Tensor:

        is_batched = src.dim() == 3
        if not self.batch_first and src.size(1) != tgt.size(1) and is_batched:
            raise RuntimeError("the batch number of src and tgt must be equal")
        elif self.batch_first and src.size(0) != tgt.size(0) and is_batched:
            raise RuntimeError("the batch number of src and tgt must be equal")

        if src.size(-1) != self.d_model or tgt.size(-1) != self.d_model:
            raise RuntimeError("the feature number of src and tgt must be equal to d_model")

        memory = self.encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask, )
        output = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=memory_mask,
                              tgt_key_padding_mask=tgt_key_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask)
        return output


    def generate_square_subsequent_mask(sz: int) -> Tensor:
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)


    def _reset_parameters(self):

        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

class TransformerEncoder(nn.Module):
    _constants_ = ['norm']

    def __init__(self, encoder_layer, num_layers, norm=None, sandwich_k=0):
        super(TransformerEncoder, self).__init__()
        self.layers = _get_clones(encoder_layer, num_layers+sandwich_k)
        self.num_layers = num_layers+sandwich_k
        self.norm = norm
        self.sandwich_k = sandwich_k

    def forward(self, src: Tensor, mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        output = src

        for i, mod in enumerate(self.layers):
            if i < self.sandwich_k:
                output = mod(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask, layer_type=1)
            elif i >= self.num_layers - self.sandwich_k:
                output = mod(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask, layer_type=2)
            else:
                output = mod(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask, layer_type=0)

        if self.norm is not None:
            output = self.norm(output)

        return output

class TransformerDecoder(nn.Module):
    _constants_ = ['norm']

    def __init__(self, decoder_layer, num_layers, norm=None, sandwich_k=0):
        super(TransformerDecoder, self).__init__()
        self.layers = _get_clones(decoder_layer, num_layers+sandwich_k)
        self.num_layers = num_layers+sandwich_k
        self.norm = norm
        self.sandwich_k = sandwich_k

    def forward(self, tgt: Tensor, memory: Tensor, tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None, tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        output = tgt

        for i, mod in enumerate(self.layers):
            if i < self.sandwich_k:
                output = mod(output, memory, tgt_mask=tgt_mask,
                      memory_mask=memory_mask,
                      tgt_key_padding_mask=tgt_key_padding_mask,
                      memory_key_padding_mask=memory_key_padding_mask, layer_type=1)
            elif i >= self.num_layers - self.sandwich_k:
                output = mod(output, memory, tgt_mask=tgt_mask,
                      memory_mask=memory_mask,
                      tgt_key_padding_mask=tgt_key_padding_mask,
                      memory_key_padding_mask=memory_key_padding_mask, layer_type=2)
            else:
                output = mod(output, memory, tgt_mask=tgt_mask,
                      memory_mask=memory_mask,
                      tgt_key_padding_mask=tgt_key_padding_mask,
                      memory_key_padding_mask=memory_key_padding_mask, layer_type=0)
            

        if self.norm is not None:
            output = self.norm(output)

        return output


class TransformerEncoderLayer(nn.Module):
    _constants_ = ['batch_first', 'norm_first']

    # layer_type = 0 means regular layer, 1 means only self-attention, 2 means only feedsforward
    def __init__(self, d_model: int, nhead: int, dim_feedforward: int = 2048, dropout: float = 0.1,
                 activation: Union[str, Callable[[Tensor], Tensor]] = F.relu,
                 layer_norm_eps: float = 1e-5, batch_first: bool = False, norm_first: bool = False,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first,
                                            **factory_kwargs)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward, **factory_kwargs)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model, **factory_kwargs)

        self.norm_first = norm_first
        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        # Legacy string support for activation function.
        if isinstance(activation, str):
            self.activation = _get_activation_fn(activation)
        else:
            self.activation = activation

    def _setstate_(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerEncoderLayer, self)._setstate_(state)

    def forward(self, src: Tensor, src_mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None\
                , layer_type=0) -> Tensor:
        x = src
        if self.norm_first:
            if layer_type == 0:
                x = x + self._sa_block(self.norm1(x), src_mask, src_key_padding_mask)
                x = x + self._ff_block(self.norm2(x))
            elif layer_type == 1:
                x = x + self._sa_block(self.norm1(x), src_mask, src_key_padding_mask)
            else:
                x = x + self._ff_block(self.norm2(x))
        else:
            if layer_type == 0:
                x = self.norm1(x + self._sa_block(x, src_mask, src_key_padding_mask))
                x = self.norm2(x + self._ff_block(x))
            elif layer_type == 1:
                x = self.norm1(x + self._sa_block(x, src_mask, src_key_padding_mask))
            else:
                x = self.norm2(x + self._ff_block(x))

        return x


    # self-attention block
    def _sa_block(self, x: Tensor,
                  attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Tensor:
        x = self.self_attn(x, x, x,
                           attn_mask=attn_mask,
                           key_padding_mask=key_padding_mask,
                           need_weights=False)[0]
        return self.dropout1(x)

    # feed forward block
    def _ff_block(self, x: Tensor) -> Tensor:
        x = self.linear2(self.dropout(self.activation(self.linear1(x))))
        return self.dropout2(x)



class TransformerDecoderLayer(nn.Module):
    _constants_ = ['batch_first', 'norm_first']

    def __init__(self, d_model: int, nhead: int, dim_feedforward: int = 2048, dropout: float = 0.1,
                 activation: Union[str, Callable[[Tensor], Tensor]] = F.relu,
                 layer_norm_eps: float = 1e-5, batch_first: bool = False, norm_first: bool = False,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first,
                                            **factory_kwargs)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first,
                                                 **factory_kwargs)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward, **factory_kwargs)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model, **factory_kwargs)

        self.norm_first = norm_first
        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.norm3 = nn.LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        # Legacy string support for activation function.
        if isinstance(activation, str):
            self.activation = _get_activation_fn(activation)
        else:
            self.activation = activation

    def _setstate_(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerDecoderLayer, self)._setstate_(state)

    def forward(self, tgt: Tensor, memory: Tensor, tgt_mask: Optional[Tensor] = None, memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None, memory_key_padding_mask: Optional[Tensor] = None\
                , layer_type=0) -> Tensor:
        # see Fig. 1 of https://arxiv.org/pdf/2002.04745v1.pdf

        x = tgt
        if self.norm_first:
            if layer_type == 0:
                x = x + self._sa_block(self.norm1(x), tgt_mask, tgt_key_padding_mask)
                x = x + self._mha_block(self.norm2(x), memory, memory_mask, memory_key_padding_mask)
                x = x + self._ff_block(self.norm3(x))
            elif layer_type == 1:
                x = x + self._sa_block(self.norm1(x), tgt_mask, tgt_key_padding_mask)
                x = x + self._mha_block(self.norm2(x), memory, memory_mask, memory_key_padding_mask)
            else:
                x = x + self._ff_block(self.norm3(x))
        else:
            if layer_type == 0:
                x = self.norm1(x + self._sa_block(x, tgt_mask, tgt_key_padding_mask))
                x = self.norm2(x + self._mha_block(x, memory, memory_mask, memory_key_padding_mask))
                x = self.norm3(x + self._ff_block(x))
            elif layer_type == 1:
                x = self.norm1(x + self._sa_block(x, tgt_mask, tgt_key_padding_mask))
                x = self.norm2(x + self._mha_block(x, memory, memory_mask, memory_key_padding_mask))
            else:
                x = self.norm3(x + self._ff_block(x))

        return x


    # self-attention block
    def _sa_block(self, x: Tensor,
                  attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Tensor:
        x = self.self_attn(x, x, x,
                           attn_mask=attn_mask,
                           key_padding_mask=key_padding_mask,
                           need_weights=False)[0]
        return self.dropout1(x)

    # multihead attention block
    def _mha_block(self, x: Tensor, mem: Tensor,
                   attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Tensor:
        x = self.multihead_attn(x, mem, mem,
                                attn_mask=attn_mask,
                                key_padding_mask=key_padding_mask,
                                need_weights=False)[0]
        return self.dropout2(x)

    # feed forward block
    def _ff_block(self, x: Tensor) -> Tensor:
        x = self.linear2(self.dropout(self.activation(self.linear1(x))))
        return self.dropout3(x)



def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


def _get_activation_fn(activation):
    if activation == "relu":
        return F.relu
    elif activation == "gelu":
        return F.gelu

    raise RuntimeError("activation should be relu/gelu, not {}".format(activation))

Pre-Trained Transformer Model

In [ ]:
import argparse
import math
import os

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

#from build_vocab import WordVocab
#from dataset import Seq2seqDataset

PAD = 0
UNK = 1
EOS = 2
SOS = 3
MASK = 4

class PositionalEncoding(nn.Module):
    "Implement the PE function. No batch support?"
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

class TrfmSeq2seq(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, n_layers, sandwich_k, sandwich_encoder, sandwich_decoder):
        super(TrfmSeq2seq, self).__init__()
        self.in_size = in_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(in_size, hidden_size)
        self.pe = PositionalEncoding(hidden_size, 0.1)
        self.sandwich_k = sandwich_k
        self.sandwich_encoder = sandwich_encoder
        self.sandwich_decoder = sandwich_decoder
        self.trfm = SandwichTransformer(d_model=hidden_size, nhead=4, 
        num_encoder_layers=n_layers, num_decoder_layers = n_layers, dim_feedforward=hidden_size,
        sandwich_k=sandwich_k, sandwich_encoder=sandwich_encoder, sandwich_decoder=sandwich_decoder,
        dropout=0.4)
        self.out = nn.Linear(hidden_size, out_size)

    def forward(self, src):
        # src: (T,B)
        embedded = self.embed(src)  # (T,B,H)
        embedded += self.pe(embedded) # (T,B,H)
        hidden = self.trfm(embedded, embedded) # (T,B,H)
        out = self.out(hidden) # (T,B,V)
        out = F.log_softmax(out, dim=2) # (T,B,V)
        return out # (T,B,V)

    def _encode(self, src):
        # src: (T,B)
        embedded = self.embed(src)  # (T,B,H)
        embedded = self.pe(embedded) # (T,B,H)
        output = embedded
        if self.sandwich_encoder:
            for i in range(self.trfm.encoder.num_layers - 1):
                if i < self.sandwich_k:
                    output = self.trfm.encoder.layers[i](output, None, layer_type=1)  # (T,B,H)
                elif i >= self.trfm.encoder.num_layers - self.sandwich_k:
                    output = self.trfm.encoder.layers[i](output, None, layer_type=2)
                else:
                    output = self.trfm.encoder.layers[i](output, None, layer_type=0)
            penul = output.cpu().detach().numpy()
            output = self.trfm.encoder.layers[-1](output, None, layer_type=2)  # (T,B,H)
        else:
            for i in range(self.trfm.encoder.num_layers - 1):
                output = self.trfm.encoder.layers[i](output, None, layer_type=0)  # (T,B,H)
            penul = output.cpu().detach().numpy()
            output = self.trfm.encoder.layers[-1](output, None, layer_type=0)  # (T,B,H)
        if self.trfm.encoder.norm:
            output = self.trfm.encoder.norm(output) # (T,B,H)
        output = output.cpu().detach().numpy()
        # mean, max, first*2
        return np.hstack([np.mean(output, axis=0), np.max(output, axis=0), output[0,:,:], penul[0,:,:] ]) # (B,4H)
    
    def encode(self, src):
        # src: (T,B)
        batch_size = src.shape[1]
        if batch_size<=100:
            return self._encode(src)
        else: # Batch is too large to load
            print('There are {:d} molecules. It will take a little time.'.format(batch_size))
            st,ed = 0,100
            out = self._encode(src[:,st:ed]) # (B,4H)
            while ed<batch_size:
                st += 100
                ed += 100
                out = np.concatenate([out, self._encode(src[:,st:ed])], axis=0)
            return out

def evaluate(model, test_loader, vocab):
    model.eval()
    total_loss = 0
    for b, sm in enumerate(test_loader):
        sm = torch.t(sm.cuda()) # (T,B)
        with torch.no_grad():
            output = model(sm) # (T,B,V)
        loss = F.nll_loss(output.view(-1, len(vocab)),
                               sm.contiguous().view(-1),
                               ignore_index=PAD)
        total_loss += loss.item()
    return total_loss / len(test_loader)

#args = parse_arguments()
# assert torch.cuda.is_available()

print('Loading dataset...')
vocab = WordVocab.load_vocab('/content/drive/MyDrive/data/vocab24.pkl') #args.vocab)
df = pd.read_csv('/content/drive/MyDrive/data/chembl_24_dropped.csv')
dataset = Seq2seqDataset(df['canonical_smiles'].values, vocab)
test_size = 10000
train, test = torch.utils.data.random_split(dataset, [len(dataset)-test_size, test_size])
train_loader = DataLoader(train, batch_size= 8 , shuffle=True, num_workers=2)   #args.batch_size:8   args.n_worker:16
test_loader = DataLoader(test, batch_size=8, shuffle=False, num_workers=2)
print('Train size:', len(train))
print('Test size:', len(test))


Loading dataset...
Train size: 1712298
Test size: 10000


In [ ]:
class baseline(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, n_layers):
        super(baseline, self).__init__()
        self.in_size = in_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(in_size, hidden_size)
        self.pe = PositionalEncoding(hidden_size, 0.1)
        
        self.trfm = nn.Transformer(d_model=hidden_size, nhead=4, 
                num_encoder_layers=n_layers, num_decoder_layers = n_layers, dim_feedforward=hidden_size)
        self.out = nn.Linear(hidden_size, out_size)

    def forward(self, src):
        # src: (T,B)
        embedded = self.embed(src)  # (T,B,H)
        embedded += self.pe(embedded) # (T,B,H)
        hidden = self.trfm(embedded, embedded) # (T,B,H)
        out = self.out(hidden) # (T,B,V)
        out = F.log_softmax(out, dim=2) # (T,B,V)
        return out # (T,B,V)

    def _encode(self, src):
        # src: (T,B)
        embedded = self.embed(src)  # (T,B,H)
        embedded = self.pe(embedded) # (T,B,H)
        output = embedded
        # if self.sandwich_encoder:
        #     for i in range(self.trfm.encoder.num_layers - 1):
        #         if i < self.sandwich_k:
        #             output = self.trfm.encoder.layers[i](output, None, layer_type=1)  # (T,B,H)
        #         elif i >= self.trfm.encoder.num_layers - self.sandwich_k:
        #             output = self.trfm.encoder.layers[i](output, None, layer_type=2)
        #         else:
        #             output = self.trfm.encoder.layers[i](output, None, layer_type=0)
        #     penul = output.cpu().detach().numpy()
        #     output = self.trfm.encoder.layers[-1](output, None, layer_type=2)  # (T,B,H)
        # else:
        #     for i in range(self.trfm.encoder.num_layers - 1):
        #         output = self.trfm.encoder.layers[i](output, None)  # (T,B,H)
        #     penul = output.cpu().detach().numpy()
        #     output = self.trfm.encoder.layers[-1](output, None)  # (T,B,H)

        for i in range(self.trfm.encoder.num_layers - 1):
            output = self.trfm.encoder.layers[i](output, None)  # (T,B,H)
            # print(f'{i} finished')
        penul = output.cpu().detach().numpy()
        output = self.trfm.encoder.layers[-1](output, None)  # (T,B,H)

        if self.trfm.encoder.norm:
            output = self.trfm.encoder.norm(output) # (T,B,H)
        output = output.cpu().detach().numpy()
        # mean, max, first*2
        return np.hstack([np.mean(output, axis=0), np.max(output, axis=0), output[0,:,:], penul[0,:,:] ]) # (B,4H)
    
    def encode(self, src):
        # src: (T,B)
        batch_size = src.shape[1]
        if batch_size<=100:
            return self._encode(src)
        else: # Batch is too large to load
            print('There are {:d} molecules. It will take a little time.'.format(batch_size))
            st,ed = 0,100
            out = self._encode(src[:,st:ed]) # (B,4H)
            # print(batch_size)
            while ed<batch_size:
                st += 100
                ed += 100
                # if ed % 1000 == 0:
                #     print(ed)
                out = np.concatenate([out, self._encode(src[:,st:ed])], axis=0)
            return out

In [ ]:
# print('Train size:', len(train))
# print('Test size:', len(test))
# print(len(vocab))

In [ ]:
model = TrfmSeq2seq(45, 256, 45, 4, 3, False,True )

In [ ]:
trfm = baseline(45, 256, 45, 4)
trfm.load_state_dict(torch.load('/content/drive/MyDrive/trfm_12_23000.pkl'))

<All keys matched successfully>

In [ ]:
for i,encoderlayer in enumerate(trfm.trfm.encoder.layers):
  model.trfm.encoder.layers[i].linear1.weight = encoderlayer.linear1.weight
  model.trfm.encoder.layers[i].linear2.weight = encoderlayer.linear2.weight
  model.trfm.encoder.layers[i].norm1.weight = encoderlayer.norm1.weight
  model.trfm.encoder.layers[i].norm2.weight = encoderlayer.norm2.weight
  model.trfm.encoder.layers[i].linear1.bias = encoderlayer.linear1.bias
  model.trfm.encoder.layers[i].linear2.bias = encoderlayer.linear2.bias
  model.trfm.encoder.layers[i].norm1.bias = encoderlayer.norm1.bias
  model.trfm.encoder.layers[i].norm2.bias = encoderlayer.norm2.bias
  model.trfm.encoder.layers[i].self_attn.in_proj_bias = encoderlayer.self_attn.in_proj_bias
  model.trfm.encoder.layers[i].self_attn.in_proj_weight = encoderlayer.self_attn.in_proj_weight

  # if i >= 2:
  #     model.trfm.encoder.layers[i+2].linear1.weight = encoderlayer.linear1.weight
  #     model.trfm.encoder.layers[i+2].linear2.weight = encoderlayer.linear2.weight
  #     model.trfm.encoder.layers[i+2].norm1.weight = encoderlayer.norm1.weight
  #     model.trfm.encoder.layers[i+2].norm2.weight = encoderlayer.norm2.weight
  #     model.trfm.encoder.layers[i+2].linear1.bias = encoderlayer.linear1.bias
  #     model.trfm.encoder.layers[i+2].linear2.bias = encoderlayer.linear2.bias
  #     model.trfm.encoder.layers[i+2].norm1.bias = encoderlayer.norm1.bias
  #     model.trfm.encoder.layers[i+2].norm2.bias = encoderlayer.norm2.bias
  #     model.trfm.encoder.layers[i+2].self_attn.in_proj_bias = encoderlayer.self_attn.in_proj_bias
  #     model.trfm.encoder.layers[i+2].self_attn.in_proj_weight = encoderlayer.self_attn.in_proj_weight
  
for j,decoderlayer in enumerate(trfm.trfm.decoder.layers):
  model.trfm.decoder.layers[j].linear1.weight = decoderlayer.linear1.weight
  model.trfm.decoder.layers[j].linear2.weight = decoderlayer.linear2.weight
  model.trfm.decoder.layers[j].norm1.weight = decoderlayer.norm1.weight
  model.trfm.decoder.layers[j].norm2.weight = decoderlayer.norm2.weight
  model.trfm.decoder.layers[j].norm3.weight = decoderlayer.norm3.weight
  model.trfm.decoder.layers[j].linear1.bias = decoderlayer.linear1.bias
  model.trfm.decoder.layers[j].linear2.bias = decoderlayer.linear2.bias
  model.trfm.decoder.layers[j].norm1.bias = decoderlayer.norm1.bias
  model.trfm.decoder.layers[j].norm2.bias = decoderlayer.norm2.bias
  model.trfm.decoder.layers[j].norm3.bias = decoderlayer.norm3.bias
  model.trfm.decoder.layers[j].self_attn.in_proj_bias = decoderlayer.self_attn.in_proj_bias
  model.trfm.decoder.layers[j].self_attn.in_proj_weight = decoderlayer.self_attn.in_proj_weight
  model.trfm.decoder.layers[j].multihead_attn.in_proj_bias = decoderlayer.multihead_attn.in_proj_bias
  model.trfm.decoder.layers[j].multihead_attn.in_proj_weight = decoderlayer.multihead_attn.in_proj_weight
  if j >= 1:
    model.trfm.decoder.layers[j+3].linear1.weight = decoderlayer.linear1.weight
    model.trfm.decoder.layers[j+3].linear2.weight = decoderlayer.linear2.weight
    model.trfm.decoder.layers[j+3].norm1.weight = decoderlayer.norm1.weight
    model.trfm.decoder.layers[j+3].norm2.weight = decoderlayer.norm2.weight
    model.trfm.decoder.layers[j+3].norm3.weight = decoderlayer.norm3.weight
    model.trfm.decoder.layers[j+3].linear1.bias = decoderlayer.linear1.bias
    model.trfm.decoder.layers[j+3].linear2.bias = decoderlayer.linear2.bias
    model.trfm.decoder.layers[j+3].norm1.bias = decoderlayer.norm1.bias
    model.trfm.decoder.layers[j+3].norm2.bias = decoderlayer.norm2.bias
    model.trfm.decoder.layers[j+3].norm3.bias = decoderlayer.norm3.bias
    model.trfm.decoder.layers[j+3].self_attn.in_proj_bias = decoderlayer.self_attn.in_proj_bias
    model.trfm.decoder.layers[j+3].self_attn.in_proj_weight = decoderlayer.self_attn.in_proj_weight
    model.trfm.decoder.layers[j+3].multihead_attn.in_proj_bias = decoderlayer.multihead_attn.in_proj_bias
    model.trfm.decoder.layers[j+3].multihead_attn.in_proj_weight = decoderlayer.multihead_attn.in_proj_weight

In [ ]:
model = model.cuda()
n_epochs=15
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * n_epochs))

In [ ]:
# from torch.optim.optimizer import Optimizer
checkpoint = torch.load('/content/drive/MyDrive/project_epoch7.pth')
model.load_state_dict(checkpoint['model_state_dict'])
# print(model.load_state_dict(checkpoint['model_state_dict']))
# print(checkpoint['model_state_dict'])

# print(optimizer)
# print(checkpoint['optimizer_state_dict']['param_groups'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 1e-05
    lr: 9.999486540617892e-06
    weight_decay: 1e-05
)
[{'lr': 5.522642316338376e-06, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 1e-05, 'amsgrad': False, 'initial_lr': 1e-05, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138]}]


ValueError: ignored

In [ ]:
# print(model)

In [ ]:
import pdb
from tqdm import tqdm
scaler = torch.cuda.amp.GradScaler()


best_loss = None
for epoch in range(12):
  print("epoch:",epoch)
  batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
  total_loss = 0.0
  for b, sm in enumerate(train_loader):
    # if sm is None
    #   continue
    sm = torch.t(sm.cuda()) # (T,B)
    optimizer.zero_grad()

    with torch.cuda.amp.autocast():
      output = model(sm) # (T,B,V)
      loss = F.nll_loss(output.view(-1, len(vocab)),sm.contiguous().view(-1), ignore_index=PAD)
    total_loss += float(loss)
    # print(float(loss))
    batch_bar.set_postfix(
            loss=f"{float(total_loss / (b + 1))}")
    
    if b>0 and b%100000 ==0:
      val_loss = evaluate(model, test_loader, vocab)
      print('Val {:3d}: iter {:5d} | loss {:.10f} | perplexity {:.10f}'.format(epoch, b, val_loss, math.exp(loss)))
      if not best_loss or loss < best_loss:
        # print("[!] saving model...")
        pathname = '/content/11785FinalProject/notebooks/project_best.pth' 
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, pathname)
        best_loss = val_loss
      pathname = '/content/11785FinalProject/notebooks/project.pth' 
      torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        }, pathname)
    
    pathname = f'/content/11785FinalProject/notebooks/project_epoch{epoch}.pth' 
    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'scheduler_state_dict': scheduler.state_dict(),
          }, pathname)
  
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    scheduler.step()
    # scheduler.step()
    batch_bar.update()
  batch_bar.close()
  # print("Epoch {}/{}: Train Loss {:.04f}".format(
  #       b + 1,
  #       b,
  #       float(total_loss / len(train_loader))))
  # torch.save({'epoch': 5,
  #       'model_state_dict': model.state_dict(),
  #       'optimizer_state_dict': optimizer.state_dict(),
  #       }, 'drive/MyDrive/project.pth')
    #print(loss)

epoch: 0


Train:  47%|████▋     | 100000/214038 [1:14:49<1:27:28, 21.73it/s, loss=2.0606740786323778e-05]

Val   0: iter 100000 | loss 0.0000170578 | perplexity 1.0000156749


Train:  93%|█████████▎| 200000/214038 [2:30:43<10:17, 22.74it/s, loss=2.0967300356839834e-05]

Val   0: iter 200000 | loss 0.0000390091 | perplexity 1.0000158437


...

Train:  47%|████▋     | 100000/214038 [1:18:57<1:28:52, 21.39it/s, loss=2.1641345322793105e-05]

Val   5: iter 100000 | loss 0.0000382563 | perplexity 1.0000304011


Train:  93%|█████████▎| 200000/214038 [2:36:24<10:46, 21.72it/s, loss=2.0690373187790542e-05]

Val   5: iter 200000 | loss 0.0000162749 | perplexity 1.0000147524


Train:  98%|█████████▊| 209174/214038 [2:43:42<03:41, 21.96it/s, loss=2.0721223014751293e-05]

In [ ]:
torch.save({
        # 'epoch': 5,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, '/content/11785FinalProject/notebooks/project.pth')

In [ ]:
df = pd.read_csv('HIV.csv')

In [ ]:
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, precision_recall_curve, auc
def evaluate_long_smiles(X_train, X_test, y_train, y_test, n_repeats):
    auc = np.empty(n_repeats)
    for i in range(n_repeats):
        clf = MLPClassifier(max_iter=1000)
        clf.fit(X_train, y_train)
        y_score = clf.predict_proba(X_test)
        auc[i] = roc_auc_score(y_test, y_score[:,1])
    ret = {}
    ret['auc mean'] = np.mean(auc)
    ret['auc std'] = np.mean(np.std(auc, axis=0))
    return ret
def evaluate_mlp_classification(X, y, rate, n_repeats):
    auc = np.empty(n_repeats)
    for i in range(n_repeats):
        clf = MLPClassifier(max_iter=1000)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-rate, stratify=y)
        clf.fit(X_train, y_train)
        y_score = clf.predict_proba(X_test)
        auc[i] = roc_auc_score(y_test, y_score[:,1])
    ret = {}
    ret['auc mean'] = np.mean(auc)
    ret['auc std'] = np.mean(np.std(auc, axis=0))
    return ret

def get_inputs(sm):
    seq_len = 220
    sm = sm.split()
    if len(sm)>218:
        print('SMILES is too long ({:d})'.format(len(sm)))
        sm = sm[:109]+sm[-109:]
    ids = [vocab.stoi.get(token, unk_index) for token in sm]
    ids = [sos_index] + ids + [eos_index]
    seg = [1]*len(ids)
    padding = [pad_index]*(seq_len - len(ids))
    ids.extend(padding), seg.extend(padding)
    return ids, seg

def get_array(smiles):
    x_id, x_seg = [], []
    for sm in smiles:
        a,b = get_inputs(sm)
        x_id.append(a)
        x_seg.append(b)
    return torch.tensor(x_id), torch.tensor(x_seg)

In [ ]:
pad_index = 0
unk_index = 1
eos_index = 2
sos_index = 3
mask_index = 4

rates = 2**np.arange(7)/80
x_split = [split(sm) for sm in df['smiles'].values]
xid, _ = get_array(x_split)
X = model.encode(torch.t(xid).cuda())
print(X.shape)
total = 0.0
for rate in rates:
  score_dic = evaluate_mlp_classification(X, df['HIV_active'].values, rate, 20)
  total += score_dict
  print(rate, score_dic)

print(total / len(rates))

SMILES is too long (220)
SMILES is too long (274)
SMILES is too long (247)
SMILES is too long (226)
SMILES is too long (244)
SMILES is too long (243)
SMILES is too long (253)
SMILES is too long (266)
SMILES is too long (346)
SMILES is too long (232)
SMILES is too long (242)
SMILES is too long (247)
SMILES is too long (240)
SMILES is too long (370)
SMILES is too long (224)
SMILES is too long (283)
SMILES is too long (265)
SMILES is too long (240)
SMILES is too long (219)
SMILES is too long (246)
SMILES is too long (243)
SMILES is too long (284)
SMILES is too long (270)
SMILES is too long (232)
SMILES is too long (260)
SMILES is too long (284)
SMILES is too long (284)
SMILES is too long (439)
SMILES is too long (491)
SMILES is too long (439)
SMILES is too long (296)
SMILES is too long (341)
SMILES is too long (285)
SMILES is too long (327)
SMILES is too long (341)
SMILES is too long (400)
SMILES is too long (263)
SMILES is too long (238)
SMILES is too long (383)
SMILES is too long (360)


Train:   0%|          | 126/214038 [00:20<2:49:40, 21.01it/s, loss=5.712149315931666]

There are 41127 molecules. It will take a little time.


KeyboardInterrupt: ignored

In [ ]:
# del model

In [ ]:
# model1 = TrfmSeq2seq(52, 256, 52, 4).cuda()
# checkpoint = torch.load('drive/MyDrive/project.pth')
# model1.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

<All keys matched successfully>

In [ ]:
# model1 = TrfmSeq2seq(45, 256, 45, 4).cpu()
# model1.load_state_dict(torch.load('/content/drive/MyDrive/trfm_12_23000.pkl'))

<All keys matched successfully>

In [ ]:
# model eval
model.eval()
total_loss = 0
for b, sm in enumerate(test_loader):
  sm = torch.t(sm.cuda()) # (T,B)
  with torch.no_grad():
      output = model(sm) # (T,B,V)
  loss = F.nll_loss(output.view(-1, len(vocab)),
                          sm.contiguous().view(-1),
                          ignore_index=PAD)
  total_loss += loss.item()
  batch_bar.set_postfix(
            loss=f"{float(total_loss / (i + 1))}")
  batch_bar.update()
batch_bar.close()
print(total_loss / len(test_loader))

NameError: ignored

In [ ]:
model1.eval()
total_loss = 0
batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Validation') 
for b, sm in enumerate(test_loader):
  sm = torch.t(sm) # (T,B)
  with torch.no_grad():
      output = model1(sm) # (T,B,V)
  loss = F.nll_loss(output.view(-1, 45),
                          sm.contiguous().view(-1),
                          ignore_index=PAD)
  total_loss += loss.item()
  batch_bar.set_postfix(
            loss=f"{float(total_loss / (b + 1))}")
  batch_bar.update()
batch_bar.close()
print(total_loss / len(test_loader))

In [ ]:
import sys
sys.path.insert(0,'/content/11785FinalProject/transformer_xl')

import data_utils
from mem_transformer import MemTransformerLM 

sys.path.insert(0,'/content/11785FinalProject/transformer_xl/utils')
from proj_adaptive_softmax import ProjectedAdaptiveLogSoftmax
#sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/11785FinalProject/transformer_xl')

In [ ]:
B = 4
tgt_len, mem_len, ext_len = 36, 36, 0
data_len = tgt_len * 20
n_token = 45
device = torch.device("cuda" if "cuda" else "cpu")
n_layer = 4
n_rel_layer = 4
n_head = 2
d_head = 2
d_model = 200
d_embed = 200
d_inner = 200
dropout = 0.0


In [ ]:
div_val = [1, 2]

cutoffs = [n_token // 2]
tie_projs = [False] + [True] * len(cutoffs)

crit = ProjectedAdaptiveLogSoftmax(n_token, d_embed, d_model, 
                                                    cutoffs, div_val=div_val)

In [ ]:
model = MemTransformerLM(n_token, n_layer, n_head,
                            d_model, d_head, d_inner, dropout,
                            dropatt=dropout, tie_weight=True, 
                            d_embed=d_embed, div_val=div_val[0], 
                            tie_projs=tie_projs, pre_lnorm=True,
                            tgt_len=tgt_len, ext_len=ext_len, mem_len=mem_len, 
                            cutoffs=cutoffs, attn_type=0).to(device)

In [ ]:
mems = tuple()
for idx, (inp, tgt, seqlen) in enumerate(train_loader):
                # print('batch {}'.format(idx))
    

from tqdm import tqdm
scaler = torch.cuda.amp.GradScaler()

for b in range(5):
  batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
  total_loss = 0.0
  for i, sm in enumerate(train_loader):
    # if sm is None:
    #   continue
    sm = torch.t(sm.cuda()) # (T,B)
    optimizer.zero_grad()

    with torch.cuda.amp.autocast():
      out = model(inp, tgt, *mems)
      mems = out[1:]
      output = out[0]
      loss = crit(output.view(-1, output.size(-1)), tgt.flatten())
      loss = loss.view(tgt.size(0), -1)
      loss = loss.float().mean().type_as(loss)
      total_loss += float(loss)
    # print(float(loss))
    batch_bar.set_postfix(
            loss=f"{float(total_loss / (i + 1))}")
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    # scheduler.step()
    batch_bar.update()
  batch_bar.close()
  print("Epoch {}/{}: Train Loss {:.04f}".format(
        b + 1,
        b,
        float(total_loss / len(train_loader))))
  torch.save({'epoch': 5,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, '/content/11785FinalProject/notebooks/project.pth')
                